<a href="https://colab.research.google.com/github/su-sumico/seminar/blob/main/Segment_Anything_Model_for_Geospatial_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segment Anything Model for Geospatial Data
* https://segment-anything.com/

In [43]:
%pip install -q segment-geospatial leafmap localtileserver

## Import libraries

In [44]:
import os
import leafmap
import torch
from samgeo import SamGeo, tms_to_geotiff

## Create an interactive map

In [45]:
m = leafmap.Map(center=[35.683117, 139.731924], zoom=10)
m.add_basemap('SATELLITE')
m

Map(center=[35.683117, 139.731924], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [46]:
#if m.user_roi_bounds() is not None:
#    bbox = m.user_roi_bounds()
#else:
#    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

In [47]:
bbox = [139.731, 35.683, 139.732, 35.684] #35.683117, 139.731924

## Download map tiles

Download maps tiles and mosaic them into a single GeoTIFF file

In [48]:
image = 'satellite.tif'
# image = '/path/to/your/own/image.tif'

In [49]:
tms_to_geotiff(output=image, bbox=bbox, zoom=10, source='Satellite', overwrite=True)

Downloaded image 1/1
Saving GeoTIFF. Please wait...
Image saved to satellite.tif


In [50]:
m.add_raster(image, layer_name='Image')
m

Map(center=[35.683117, 139.731924], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Initialize SAM class

In [51]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

In [52]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    checkpoint=checkpoint,
    model_type='vit_h',
    device=device,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=None,
)

## Segment the image

In [53]:
mask = 'segment.tiff'
sam.generate(image, mask)

## Polygonize the raster data

Save the segmentation results as a GeoPackage file.

In [54]:
vector = 'segment.gpkg'
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

You can also save the segmentation results as any vector data format supported by GeoPandas.

In [55]:
shapefile = 'segment.shp'
sam.tiff_to_vector(mask, shapefile)

## Visualize the results

In [57]:
 style = {
     'color': '#9900ff',
     'weight': 2,
     'fillColor': '#7c4199',
     'fillOpacity': 0.35,
 }
 m.add_vector(vector, layer_name='Vector', style=style)
 m

Map(bottom=13212823.0, center=[35.68536561864818, 139.73444223403934], controls=(ZoomControl(options=['positio…